In [ ]:
import numpy as np
import pandas as pd
import re
import time
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
raw = pd.read_csv('raw_report.csv')

In [ ]:
raw.head()

In [ ]:
raw.loc[15565,]

In [ ]:
raw = raw.drop(index=15565)

In [ ]:
raw.dtypes

In [ ]:
dropList = [
    2062896,    7473118,    1722390,        6794102,    2317395,    2149084,    29572788,    3965629,    34467296,
    2922840,    2879700,    38680925,    18901396,    2782099,    5820834,    36565513,    7444317,    2851588,
    37271324,    5328272,    1999912,    23391049,    33190257,    5908857,    22846563,    38030852,    31054720,
    1945386,    36758173,    37045153,    18493342,    39249867,    29770945,    1833912,    33012109,    32108530,
    23008973,    34047208,    17289337,    1892914,    36273475,    37312913,    36245592,    7507894,    32928435,
    4503562,    20363070,    29344325,    5779893,    31985520,    32358617,    32034901,    2760006,    1639532,
    24961539,    37977984,    5802971,    4634115,    39250157,    32928397,    33490536,    30045940,    4929972,
    36094645,    6835327,    37586409,    39199121,    32256818,    1704242,    3798636,    17212789,    4410883,
    6385249,    4433455,    2055864,    4949358,    5912724,    15837672,    36325835,    33445471,    36618229,
    4362213,    33956835,    32499331,    36325509,    36868257,    32061452,    15285712,    36115975,    32059951,
    36860036,    32481177,    32002920,    34431545,    36656636,    36325301,    34197726,    33129443,    5796048,
    34303712,    32389983,    34248344,    37256129,    36173758,    36638493,    36744373,    32841709,    20307399,
    23716990,    2017834,    39095394,    39199516,    2055869,    2055877,    39225250,    37758302,    17201891,
    2327318,    37508718,    17201438,    7491251,    23561779,    39377545,    7491251,    23561779,    39377545,
    39108711,    18228907,    20348425,    25002999,    38190739,    38043997,    36131692,    18448844,
]


In [ ]:
e = []
for i in raw.index:
    if raw.loc[i,'Id'] in dropList:
        e.append(i)

In [ ]:
raw = raw.drop(index=e)

In [ ]:
raw.shape

In [ ]:
raw = raw.rename(index=str, columns={'Answer (ignore the character limit)' : 'Answer',
                                    'Notes: (e.g., full question, sources consulted / recommended, problems, etc.)' : 'Notes FULL',
                                    'Who answered (see list of initials​)' : 'Who answers',
                                    'READ Scale (visit READ Scale site)':'READ',
                                   })

In [ ]:
time_spent = ['0-1','2-5','5-Feb','10-Jun','20-Nov','21-60','61+']
time_spent_replace = ['0-1 min','2-5 min','2-5 min','6-10 min','11-20 min','21-60 min','61+ min']
raw['Time Spent'] = raw['Time Spent'].replace(to_replace=time_spent, value=time_spent_replace)

In [ ]:
faq = pd.read_csv('frequentQuestions.csv', header=None)

In [ ]:
for i in raw.index:
    if raw.loc[i,'Question'] in list(faq.loc[:,0]):
        raw.loc[i,'Question'] = 'NULL'

In [ ]:
raw = raw.fillna('NULL')

In [ ]:
raw.loc[raw['READ']=='unknown','READ'] = "Unknown"

In [ ]:
raw.loc[(raw['Question Type']=='NULL')|(raw['Question Type']=='unknown'),'Question Type'] = 'Unknown'
raw.loc[raw['Question Type']=='Research Consultation','Question Type'] = 'ResearchConsultation'
raw.loc[raw['Question Type']=='K-REx','Question Type'] = 'KREx'
raw.loc[raw['Question Type']=='New Prairie Press','Question Type'] = 'NewPrairiePress'
raw.loc[raw['READ']=='NULL','READ'] = 'Unknown'

In [ ]:
raw['Question Type'].value_counts()

In [ ]:
raw.loc[raw['READ'] == '10-Jun','Id']

In [ ]:
raw.loc[raw['READ'] == '10-Jun','READ'] = 'NULL'
raw.loc[raw['READ'] == '1 (minimal effort)','READ'] = '1'
raw.loc[raw['READ'] == '6 (maximal effort)','READ'] = '6'
raw['Id'] = raw['Id'].astype(int)

In [ ]:
raw['READ_1_vs_2'] = int(0)
raw['READ_2_vs_3'] = int(0)

In [ ]:
for i in raw.index:
    if raw.loc[i,'READ'] in ['3','4','5','6']:
        raw.loc[i,'READ_1_vs_2'] = int(1)
        raw.loc[i,'READ_2_vs_3'] = int(1)
        
    if raw.loc[i,'READ'] in ['2']:
        raw.loc[i,'READ_1_vs_2'] = int(1)
        raw.loc[i,'READ_2_vs_3'] = int(0)
        
    if raw.loc[i,'READ'] in ['1']:
        raw.loc[i,'READ_1_vs_2'] = int(0)
        raw.loc[i,'READ_2_vs_3'] = int(0)
        
    if raw.loc[i,'READ'] in ['Unknown']:
        raw.loc[i,'READ_1_vs_2'] = int(999)
        raw.loc[i,'READ_2_vs_3'] = int(999)
        
    if raw.loc[i,'READ'] in ['unknown']:
        raw.loc[i,'READ_1_vs_2'] = int(999)
        raw.loc[i,'READ_2_vs_3'] = int(999)

In [ ]:
# raw[['READ','READ_1_vs_2','READ_2_vs_3']].head(50)

In [ ]:
raw['READ'].value_counts()

In [ ]:
pd.Categorical(raw['READ']).describe()

In [ ]:
raw.head()

In [ ]:
raw['Id'] = raw['Id'].astype(int)

In [ ]:
raw['Question'] = raw['Question'].astype(str)
raw['Answer'] = raw['Answer'].astype(str)
raw['Notes FULL'] = raw['Notes FULL'].astype(str)
raw['Transcript'] = np.NaN
raw['TransLength'] = np.NaN

In [ ]:
raw['Start Date'].head()

In [ ]:
raw['Start Date'] = pd.to_datetime(raw['Start Date'])
raw['End Date'] = pd.to_datetime(raw['End Date'])
raw['Start Time'] = pd.to_datetime(raw['Start Time']).dt.time

In [ ]:
raw['Start Date'].head()

In [ ]:
raw.head()

In [ ]:
raw.columns

In [ ]:
raw.shape

In [ ]:
raw = raw.drop_duplicates(subset=['Question','Answer','Question Type','READ','tags','Time Spent'])

In [ ]:
raw.shape

In [ ]:
pd.Series(raw['Question'].value_counts())[:10]

In [ ]:
raw.dtypes

In [ ]:
for i in raw.index:
    a = raw.loc[i,'Question']
    b = raw.loc[i,'Answer']
    c = raw.loc[i,'Notes FULL']
    
    la = len(a)
    lb = len(b)
    lc = len(c)
    
    k = 'null'
    if (la > lb) & (la > lc):
        k = a
    elif (lb > la) & (lb > lc):
        k = b
    else:
        k = c
    raw.loc[i,'Transcript'] = k
    raw.loc[i,'TransLength'] = len(k)
    print(i,la,lb,lc,len(k))

In [ ]:
raw.head()

In [ ]:
raw.shape

In [ ]:
raw['TransLength'].quantile(.1)

In [ ]:
lengthFilter = raw.loc[((raw['TransLength'] > raw['TransLength'].quantile(.1)) & 
                    (raw['TransLength'] < raw['TransLength'].quantile(.9))),].copy()

In [ ]:
lengthFilter.head()

In [ ]:
lengthFilter['TransLength'].describe()

In [ ]:
raw.loc[raw['Id']==24412048,'Transcript'] = """     sample redacted     """
raw.loc[raw['Id']==2124740,'Transcript'] = """     sample redacted     """
raw.loc[raw['Id']==24489117,'Transcript'] = """     sample redacted     """
raw.loc[raw['Id']==1730574,'Transcript'] = """     sample redacted     """

In [ ]:
filter_list = list(lengthFilter.loc[(lengthFilter['tags'].str.contains('patron - very happy') |
                            lengthFilter['tags'].str.contains('patron - very unhappy')),:].index)

In [ ]:
pd.Categorical(lengthFilter.loc[filter_list,'READ']).describe()

In [ ]:
pd.Categorical(lengthFilter.loc[filter_list,'Question Type']).describe()

In [ ]:
def newSegmenter(df):
    df = df.copy()
    
    seg = []
    
    for i in df.index:
        t = df.loc[i,'Transcript']
        
        seglist = []
            
        if re.match(pattern="^\d{1,2}:\d{2}\s\S{6,}\s|^\d{1,2}:\d{2}\sme\s",string=t):
            nt = re.split(pattern="\s(?=\d{1,2}:\d{2}\s\S{6,}\s)|\s(?=\d{1,2}:\d{2}\sme\s)", string=t)
            for k in nt:
                seglist.append(re.split(pattern="\s",string=k,maxsplit=2))
            
            seg.append(seglist)
        else:
            seg.append('')
            
    return(seg)

In [ ]:
raw['segTrans'] = newSegmenter(raw)
raw['segCode'] = int()

In [ ]:
newSegmenter(raw.loc[raw['Id']==18883437,])

In [ ]:
print(raw.loc[raw['Id']==18883437,'Transcript'][0])

In [ ]:
raw.loc[raw.index==39108711,]

In [ ]:
for i in raw.index:
    if raw.loc[i,'segTrans'] == '':
        raw.loc[i,'segCode'] = 1
    else:
        raw.loc[i,'segCode'] = 0

In [ ]:
raw.loc[raw['Id']==17171207,]

In [ ]:
raw['segCode'].value_counts()

In [ ]:
def referralSegmenter(df):
    df = df.copy()
    
    df = df.loc[df['Transcript'].str.startswith("Referring URL: ") & df['segCode'] == 1,].copy()
    
    seg = pd.Series()
    
    for i in df.index:
        seglist = []
       
        try:
            t = re.split(pattern='\s', string=df.loc[i,'Transcript'], maxsplit=3)[3]
        except:
            continue
            
        if re.match(pattern="^\d{1,2}:\d{2}\s\S{6,}\s",string=t):
            nt = re.split(pattern="\s(?=\d{1,2}:\d{2}\s\d{5,}\s)|\s(?=\d{1,2}:\d{2}\sme\s)", string=t)
            for k in nt:
                seglist.append(re.split(pattern="\s",string=k,maxsplit=2))
        else:
            continue
            
        seg.loc[i] = seglist
            
    return(seg)

In [ ]:
nr = referralSegmenter(raw)
raw.loc[nr.index,'segTrans'] = nr

for i in raw.index:
    if raw.loc[i,'segTrans'] == '':
        raw.loc[i,'segCode'] = 1
    else:
        raw.loc[i,'segCode'] = 0

In [ ]:
raw.loc[raw['segCode']==1,].shape

In [ ]:
raw['segCode'].value_counts()

In [ ]:
def errorTimeSegmenter(df):
    df = df.copy()
    
    df = df.loc[df['Transcript'].str.startswith(":") & df['segCode'] == 1,].copy()
    
    seg = pd.Series()
    
    for i in df.index:
        seglist = []

        t = df.loc[i,'Transcript']
        if re.match(pattern="^:\d{2}\s\S{6,}\s",string=t):
            nt = re.split(pattern="\s(?=\d{1,2}:\d{2}\s\d{5,}\s)|\s(?=\d{1,2}:\d{2}\sme\s)", string=t)
            for k in nt:
                seglist.append(re.split(pattern="\s",string=k,maxsplit=2))
        else:
            continue


        seg.loc[i] = seglist
            
    return(seg)

In [ ]:
net = errorTimeSegmenter(raw)
raw.loc[net.index,'segTrans'] = net

for i in raw.index:
    if raw.loc[i,'segTrans'] == '':
        raw.loc[i,'segCode'] = 1
    else:
        raw.loc[i,'segCode'] = 0

In [ ]:
t = "12:11 78578940005542826253835712 Hi I am wond"

In [ ]:
if re.match(pattern="^\d{0,2}:\d{2}\s\S{6,}\s", string=t):
    print(True)
else:
    print(False)

In [ ]:
raw.loc[raw['segCode']==1,['Question','Answer','Transcript'],]  

In [ ]:
def recombinerSegmenters(df):
    df = df.copy()
    df = df.loc[df['segCode']==1,]
    seg = pd.Series()
    for i in df.index:
        seglist = []
        if re.match(pattern="^\d{0,2}:\d{2}\s\S{6,}\s", string=df.loc[i,'Question']):
            if re.match(pattern="^\d{0,2}:\d{2}\sme\s", string=df.loc[i,'Transcript']):
                t = df.loc[i,'Question'] + ' ' + df.loc[i,'Transcript']
                
                if re.match(pattern="^\d{0,2}:\d{2}\s\S{6,}",string=t):
                    nt = re.split(pattern="\s(?=\d{1,2}:\d{2}\s\S{6,}\s)|\s(?=\d{1,2}:\d{2}\sme\s)", string=t)
                    for k in nt:
                        seglist.append(re.split(pattern="\s",string=k,maxsplit=2))
        
        seg.loc[i] = seglist
             
                
                
    return(seg)

In [ ]:
nrs = recombinerSegmenters(raw)
nrs = nrs[nrs.str.len() > 0].copy()

raw.loc[nrs.index,'segTrans'] = nrs

for i in raw.index:
    if raw.loc[i,'segTrans'] == '':
        raw.loc[i,'segCode'] = 1
    else:
        raw.loc[i,'segCode'] = 0

In [ ]:
raw['segCode'].value_counts()

In [ ]:
raw.columns

In [ ]:
raw.Transcript.duplicated().value_counts()

In [ ]:
print(raw.loc['10158','Transcript'])
print('\n')
print(raw.loc['10158','Question'])

In [ ]:
print(raw.loc['10159','Transcript'])
print(raw.loc['10159','Question'])

In [ ]:
raw.loc[raw.duplicated(subset=['Transcript','Question','Answer'],keep=False),['Id','Time Spent','tags','READ','Question Type','Answer','Question','Transcript']].sort_values(by=['Transcript'])

In [ ]:
raw = raw.drop(index=raw.loc[raw['Transcript']=='3rd floor noise complaint'].index)
raw = raw.drop(index=raw.loc[raw['Transcript']=='Noise complaint'].index)
raw = raw.drop(index=raw.loc[raw['Transcript']=='Scanner help'].index)
raw = raw.drop(index=raw.loc[raw['Transcript']=='noise complaint'].index)
raw = raw.drop(index=raw.loc[raw['Transcript']=='room reservation'].index)
raw = raw.drop(index=raw.loc[raw['Transcript']=='HLH - FILLER'].index)
raw = raw.drop(index=raw.loc[raw['Transcript']=='Do you have this resource (article, book, textbook, etc.)?'].index)
raw = raw.drop(index=raw.loc[raw['Transcript']=='University Experience Scavenger Hunt'].index)
raw = raw.drop(index=raw.loc[raw['Transcript']=='HLH - Do you have this book (or textbook)?'].index)
raw = raw.drop(index=raw.loc[raw['Transcript']=='NULL'].index)

In [ ]:
raw.loc[raw.duplicated(subset=['Transcript'],keep=False),['Id','Time Spent','tags','READ','Question Type','Answer','Question','Transcript']].sort_values(by=['Transcript'])

In [ ]:
raw['segCode'].value_counts()

In [ ]:
raw.columns

In [ ]:
def weirdTimeSegmenter(df):
    df = df.copy()
    
    df = df.loc[df['segCode']==1,]
    df = df.loc[df['Transcript'].str.contains("^\d{1,2}:\d{2}\D{1,2}\s\S{6,}\S{5,}", regex=True),]
    
    seg = pd.Series()
    
    for i in df.index:
        seglist = []
        t = df.loc[i,'Transcript']
            
#         try:
        if re.match(pattern="^\d{1,2}:\d{2}\D{1,2}\s\S{6,}\S{5,}",string=t):
            nt = re.split(pattern="\s(?=\d{1,2}:\d{2}\D{2}\s\S{5,})", string=t)
            for k in nt:
                seglist.append(re.split(pattern="\s",string=k,maxsplit=2))
                
        else:
            continue
#         except:
#             seglist.append('Error')

        seg.loc[i] = seglist
            
    return(seg)

In [ ]:
raw.loc['6119','Transcript'] = """02:41PM 47187626333404367048001996@web.libraryh3lp.com how annoying is it to listen to the beeping for chats all day long? We were just told that you hear a beep until you pick up the chat. 02:39PM hale-ask-a-librarian@chat.libraryh3lp.com Some folks work more hours a day on it than others. I am one of many who do it only a few hours a week. It doesn\'t really bother me because it calls my attention away from what I am doing and alerts me that I need to pay attention to the question coming in. However, I can transfer this question to someone who works many more hours a week than I do if you would like to get a 2nd opinion. 02:39PM hale-ask-a-librarian@chat.libraryh3lp.com The chats are normally spread out enough that they don\'t pile on top of each other. When they do, just having to juggle 2-4 at once is a challenge. 02:41PM 47187626333404367048001996@web.libraryh3lp.com Do you ever get confused on which chat to answer first? 02:42PM hale-ask-a-librarian@chat.libraryh3lp.com A little. The new chat boxes can literally hide behind one you have open, so you have to click and drag to expose the new question. 02:44PM hale-ask-a-librarian@chat.libraryh3lp.com However, as you should see, there are times embedded in the chat so we can see who is first. We try to quickly answer Hello to let somone know that we are here. I found when I try to say much more besides Hello to begin with, another person monitoring may think the chat isn\'t being picked up and will take it to make sure the question is being answered. 02:44PM hale-ask-a-librarian@chat.libraryh3lp.com There are times you are immersed in truly difficult questions, so that is a challenge when multiple chats hit at once. 02:45PM 47187626333404367048001996@web.libraryh3lp.com What is the weirdest thing anyone has asked you on the chats? 02:45PM hale-ask-a-librarian@chat.libraryh3lp.com Oh my. Let me think; there have been some odd things. 02:47PM hale-ask-a-librarian@chat.libraryh3lp.com At the moment there isn\'t anything that pops up first, but we do occasionally get what I assume are guys who get fresh and want a date or something. I tell them that they probably aren\'t interested in anyone with 6 grandchildren. 02:50PM 47187626333404367048001996@web.libraryh3lp.com My mom uses the phrase "get fresh"! That must be crazy. They have no idea who they are even talking to! 02:51PM hale-ask-a-librarian@chat.libraryh3lp.com Very true! 02:51PM hale-ask-a-librarian@chat.libraryh3lp.com There are men who answer chat also, and that might surprise some folks. 02:54PM hale-ask-a-librarian@chat.libraryh3lp.com I try to make an effort to be friendly and show genuine empathy. As we were chatting another question came in about a book that is on Reserves. I told the person that it isn\'t checked out at the moment and wished them good luck on the assignment. If it is a cold day, I\'ll sometimes say something about staying warm, or if it is horrifically hot, maybe say something about staying cool. I think it humanizes the chat service, and we need to make those "I care" connections with the patrons. 02:55PM hale-ask-a-librarian@chat.libraryh3lp.com I wish I could remember a crazy chat at the moment, because I know I\'ve had a few strange one. 02:55PM hale-ask-a-librarian@chat.libraryh3lp.com ones. 02:56PM 47187626333404367048001996@web.libraryh3lp.com That is an amazing idea! I always feel like I am chatting with a computer; and I think making that connection is the real difference in making someone feel comfortable with the library and the "ask a librarian" feature. 02:56PM hale-ask-a-librarian@chat.libraryh3lp.com We have a phone line, too, and for a while there was a guy from Colorado who would call and ask us to look up certain websites. I realized he was listening to late night radio--Coast to Coast a.m.--and he wanted to know things ranging from politcal conspiracy theories to alien landings. 02:57PM hale-ask-a-librarian@chat.libraryh3lp.com I\'m glad we can help you feel connected. There really are human beings answering our chat. 02:57PM 47187626333404367048001996@web.libraryh3lp.com Thank you so much! I hope you have a beautiful day. Even though the sky looks gloomy; it is beautiful out there! I enjoyed this conversation. 02:57PM hale-ask-a-librarian@chat.libraryh3lp.com It wasn\'t an IM. He didn\'t have a computer so he would call us. We all got to know his voice really we.. 02:57PM hale-ask-a-librarian@chat.libraryh3lp.com well. 02:58PM hale-ask-a-librarian@chat.libraryh3lp.com Thank you! I also you have a terrific day! I also enjoyed the conversation. People seldom ask these kinds of questions. 02:58PM hale-ask-a-librarian@chat.libraryh3lp.com Take care! 03:01PM 47187626333404367048001996@web.libraryh3lp.com Hey Hale! 03:02PM hale-ask-a-librarian@chat.libraryh3lp.com I haven\'t closed my window--want to talk to someone else? 03:02PM hale-ask-a-librarian@chat.libraryh3lp.com I assume you are the one I\'ve talked to a few minutes. 03:02PM 47187626333404367048001996@web.libraryh3lp.com Oh! Yes! I am totally good talking with you again! I was just wondering what job opportunities are offered to students through the library? 03:03PM hale-ask-a-librarian@chat.libraryh3lp.com we have a variety of things. We have several who work in user services and are at the help desk. We have students who work in Archives and Special Collections, and we have students who work for Building Services. 03:03PM hale-ask-a-librarian@chat.libraryh3lp.com Let me try to find the application instructions. 03:04PM hale-ask-a-librarian@chat.libraryh3lp.com http://www.lib.k-state.edu/jobs 03:04PM hale-ask-a-librarian@chat.libraryh3lp.com Scroll down to Student positions. 03:04PM hale-ask-a-librarian@chat.libraryh3lp.com Oh--we also have students in Interlibrary Services. 03:05PM hale-ask-a-librarian@chat.libraryh3lp.com I\'m sure there are more places, but those are what I\'m thinking of off the top of my head. 03:06PM 47187626333404367048001996@web.libraryh3lp.com Thank you so much, again! Have a wonderful day! 03:06PM hale-ask-a-librarian@chat.libraryh3lp.com You too!"""
raw.loc['10730','Transcript'] = """01:25PM 76952912671820322253094861@web.libraryh3lp.com is the KSU database the same as the following link? It is for a new class but if its the same I will disregard and use the database resource? thanks 06:25PM 76952912671820322253094861@web.libraryh3lp.com https://k-state.instructure.com/courses/38718/external_tools/3399 01:26PM hale-ask-a-librarian@chat.libraryh3lp.com hi 01:26PM hale-ask-a-librarian@chat.libraryh3lp.com I'm checking ... be right there 01:27PM hale-ask-a-librarian@chat.libraryh3lp.com ok, so that is a research guide that is embedded into KState online/canvas for you to use for class 01:28PM hale-ask-a-librarian@chat.libraryh3lp.com I can't see it because I'm not a member of the class, but yes, it should link you to everything that you would get to from the library's website 01:28PM 76952912671820322253094861@web.libraryh3lp.com what makes it different from using ksu database for peer review and articles? 01:28PM 76952912671820322253094861@web.libraryh3lp.com ok thanks. I am just trying to unclog my mind with unnecessary resources. thanks 01:28PM hale-ask-a-librarian@chat.libraryh3lp.com I understand .... this semester was the first semester that we put these in KState Online 01:29PM hale-ask-a-librarian@chat.libraryh3lp.com we thought it might be easier if there was just one place you had to go 01:29PM 76952912671820322253094861@web.libraryh3lp.com ok thanks a bunch, I am trying not to learn "new" things if I don't have to since I am at the end of my journey. 01:29PM hale-ask-a-librarian@chat.libraryh3lp.com So you get the class info and then the research info without going out to a different website 01:29PM hale-ask-a-librarian@chat.libraryh3lp.com Understandable ... it's all the same so use whichever you are comfortable/familiar with 01:30PM hale-ask-a-librarian@chat.libraryh3lp.com ok, have a good day 01:30PM 76952912671820322253094861@web.libraryh3lp.com thanks! :) you too"""
raw.loc['362','Transcript'] = """2:59 980725465186257027863508592 Hi there, I was wondering if the library computers had Windows Movie Maker or some other sort of video editing program on them? 2:59 me Hi 2:59 me The computers in the Media Development Center have some editing software 3:00 me Actually there's several programs - I'm not certain if Movie Maker is among them, but they have several options 3:00 me That's on the 2nd floor with iTAC 3:01 98072546518625702786350859 Okay, thank you so much for your help 3:01 me You're very welcome! 2:59 me Hi 2:59 me The computers in the Media Development Center have some editing software 3:00 me Actually there's several programs - I'm not certain if Movie Maker is among them, but they have several options 3:00 me That's on the 2nd floor with iTAC 3:01 98072546518625702786350859 Okay, thank you so much for your help 3:01 me You're very welcome!"""

In [ ]:
weirdTimeSegmenter(raw)

In [ ]:
nws = weirdTimeSegmenter(raw)

raw.loc[nws.index,'segTrans'] = nws

for i in raw.index:
    if raw.loc[i,'segTrans'] == '':
        raw.loc[i,'segCode'] = 1
    else:
        raw.loc[i,'segCode'] = 0

In [ ]:
raw['segCode'].value_counts()

In [ ]:
raw = raw.drop(index=raw.loc[raw['TransLength'] <100,].index)

In [ ]:
raw.loc[(raw['TransLength'] <100) &
        (raw['segCode']==1),].shape

In [ ]:
raw.loc[raw['segCode']==1,].shape

In [ ]:
raw.loc[(raw['segCode']==1) & 
                 (raw['TransLength'] < 5000) & 
                 ~(raw['Transcript'].str.contains("\d{5,}",regex=True)) &
                 ~(raw['Question'].str.contains("\d{5,}",regex=True)),].shape

In [ ]:
raw.loc[raw['segCode']==1,'TransLength'].describe()

In [ ]:
def guestIPsegmenter(df):
    df = df.copy()
    
    df = df.loc[df['Transcript'].str.contains(". The guest's IP is ", regex=False),]
    
    seg = pd.Series()
    
    for i in df.index:
        seglist = []
        t = df.loc[i,'Transcript']
        
        t = re.split(string=t,pattern="The guest\'s IP is\s\S{6,}\s")[1]    
        
        t = re.sub(pattern="(?<=Guest)\s(?=\d{2,})|(?<=\d{2})\s(?=PM|AM)", repl="",string=t)
        t = re.sub(pattern="(?<=\d{2}(AM|PM))\sfrom\s(?=Guest)|(?<=\d{2}(AM|PM))\sfrom\s(?=me\s)", repl=" ", string=t)
        
        if re.match(pattern="^\d{1,2}:\d{2}\D{2}\sGuest",string=t):
            nt = re.split(pattern="\s(?=\d{1,2}:\d{2}\D{2}\sGuest)|\s(?=\d{1,2}:\d{2}\D{2}\sme)", string=t)
            for k in nt:
                seglist.append(re.split(pattern="\s",string=k,maxsplit=2))

        else:
            continue

        seg.loc[i] = seglist
            
    return(seg)

In [ ]:
guestIPsegmenter(raw)

In [ ]:
nwip = guestIPsegmenter(raw)

raw.loc[nwip.index,'segTrans'] = nwip

for i in raw.index:
    if raw.loc[i,'segTrans'] == '':
        raw.loc[i,'segCode'] = 1
    else:
        raw.loc[i,'segCode'] = 0

In [ ]:
def textConvo(df):
    df = df.copy()
    
    df = df.loc[df['Transcript'].str.contains("\d{2}\s\+\d{4,}", regex=True),]
    
    seg = pd.Series()
    
    for i in df.index:
        seglist = []
        t = df.loc[i,'Transcript']
        
        if re.match(pattern="^\d{1,2}:\d{2}\s\+\d{7,}\s",string=t):
            nt = re.split(pattern="\s(?=\d{1,2}:\d{2}\s\+\d{7,})|\s(?=\d{1,2}:\d{2}\sme)", string=t)
            for k in nt:
                seglist.append(re.split(pattern="\s",string=k,maxsplit=2))

        else:
            continue

        seg.loc[i] = seglist
            
    return(seg)

In [ ]:
textConvo(raw).head()

In [ ]:
ntxt = textConvo(raw)

raw.loc[ntxt.index,'segTrans'] = ntxt

for i in raw.index:
    if raw.loc[i,'segTrans'] == '':
        raw.loc[i,'segCode'] = 1
    else:
        raw.loc[i,'segCode'] = 0

In [ ]:
print(raw.loc[raw['segCode']==1,].shape)
print(raw.loc[raw['segCode']==0,].shape)

In [ ]:
raw.loc[raw['Id']==17171207,]

In [ ]:
def newSegmenterUsernames(df):
    df = df.copy()
    df = df.loc[df['segCode']==1,]
    df = df.loc[df['Transcript'].str.contains("^\d{1,2}:\d{2}\s\d{6,}", regex=True),]
    
    seg = pd.Series()
    
    for i in df.index:
        seglist = []
        t = df.loc[i,'Transcript']
        
        if re.match(pattern="^\d{1,2}:\d{2}\s\d{6,}",string=t):
            nt = re.split(pattern="\s(?=\d{0,2}:\d{2}\s\S{2,})", string=t)
            for k in nt:
                seglist.append(re.split(pattern="\s",string=k,maxsplit=2))

        else:
            continue

        seg.loc[i] = seglist
            
    return(seg)

In [ ]:
newSegmenterUsernames(raw).head()

In [ ]:
nseg = newSegmenterUsernames(raw)

raw.loc[nseg.index,'segTrans'] = nseg

for i in raw.index:
    if raw.loc[i,'segTrans'] == '':
        raw.loc[i,'segCode'] = 1
    else:
        raw.loc[i,'segCode'] = 0

In [ ]:
print(raw.loc[raw['segCode']==1,].shape)
print(raw.loc[raw['segCode']==0,].shape)

In [ ]:
def sendFileBlock(df):
    df = df.copy()
    
    df = df.loc[df['Transcript'].str.contains(" DON'T DELETE ME! ", regex=False),]
    df = df.loc[df['Transcript'].str.contains(" Send File Block ", regex=False),]
    
    seg = pd.Series()
    
    for i in df.index:
        seglist = []
        t = df.loc[i,'Transcript']
        
        t = re.split(string=t,pattern=" Send File Block ")[1]    
        
        if re.match(pattern="^\d{1,2}:\d{2}\s\S{2,}",string=t):
            nt = re.split(pattern="\s(?=\d{1,2}:\d{2}\s\S{5,}\s)|\s(?=\d{1,2}:\d{2}\sme\s)", string=t)
            for k in nt:
                seglist.append(re.split(pattern="\s",string=k,maxsplit=2))

        else:
            continue

        seg.loc[i] = seglist
            
    return(seg)

In [ ]:
sendFileBlock(raw)

In [ ]:
nblock = sendFileBlock(raw)

raw.loc[nblock.index,'segTrans'] = nblock

for i in raw.index:
    if raw.loc[i,'segTrans'] == '':
        raw.loc[i,'segCode'] = 1
    else:
        raw.loc[i,'segCode'] = 0

In [ ]:
print(raw.loc[raw['segCode']==1,].shape)
print(raw.loc[raw['segCode']==0,].shape)

In [ ]:
def strangeGuest(df):
    df = df.copy()
    
    df = df.loc[df['Transcript'].str.contains(" Guest \d{3,} ", regex=True),]
    
    seg = pd.Series()
    
    for i in df.index:
        seglist = []
        t = df.loc[i,'Transcript']
        if re.match(pattern="^This chat is from ",string=t):
            t = re.split(string=t,pattern="\sfrom\s\S{4,}\s{1}")[1]
        
        if re.match(pattern="^From ",string=t):
            t = re.split(string=t,pattern="rom\s\S{2,}\s")[1]  
        
        t = re.sub(pattern="(?<=Guest)\s(?=\d{2,})|(?<=\d{2})\s(?=PM|AM)", repl="",string=t)
        t = re.sub(pattern="(?<=\d{2}(AM|PM))\sfrom\s(?=Guest)|(?<=\d{2}(AM|PM))\sfrom\s(?=me\s)", repl=" ", string=t)
        
        if re.match(pattern="^\d{1,2}:\d{2}\D{2}\sGuest",string=t):
            nt = re.split(pattern="\s(?=\d{1,2}:\d{2}\D{2}\sGuest)|\s(?=\d{1,2}:\d{2}\D{2}\s\S{2,}\s)", string=t)
            for k in nt:
                seglist.append(re.split(pattern="\s",string=k,maxsplit=2))

        else:
            continue


        seg.loc[i] = seglist
            
    return(seg)

In [ ]:
strangeGuest(raw)

In [ ]:
strg = strangeGuest(raw)

raw.loc[strg.index,'segTrans'] = strg

for i in raw.index:
    if raw.loc[i,'segTrans'] == '':
        raw.loc[i,'segCode'] = 1
    else:
        raw.loc[i,'segCode'] = 0

In [ ]:
print(raw.loc[raw['segCode']==1,].shape)
print(raw.loc[raw['segCode']==0,].shape)

In [ ]:
def questionCombinerSimple(df):
    df = df.copy()
    
    df = df.loc[(df['segCode']==1) &
                 (df['Question'].str.contains("^\d{1,2}:\d{2}", regex=True)) &
                 (df['Transcript'].str.contains("^\d{0,2}:{0,1}\d{1,2}", regex=True))
                 ,]
    
    seg = pd.Series()
    
    for i in df.index:
        seglist = []
        t = df.loc[i,'Question'] + ' ' + df.loc[i,'Transcript']
        
        if re.match(pattern="^\d{1,2}:\d{2}",string=t):
            nt = re.split(pattern="\s(?=\d{1,2}:\d{2}\s\S{4,}\s)|\s(?=\S{0,3}\d{2}\sme\s)", string=t)
            for k in nt:
                seglist.append(re.split(pattern="\s",string=k,maxsplit=2))

        else:
            continue


        seg.loc[i] = seglist
            
    return(seg)

In [ ]:
questionCombinerSimple(raw)

In [ ]:
qcsimp = questionCombinerSimple(raw)

raw.loc[qcsimp.index,'segTrans'] = qcsimp

for i in raw.index:
    if raw.loc[i,'segTrans'] == '':
        raw.loc[i,'segCode'] = 1
    else:
        raw.loc[i,'segCode'] = 0

In [ ]:
print(raw.loc[raw['segCode']==1,].shape)
print(raw.loc[raw['segCode']==0,].shape)

In [ ]:
def questionCombinerAdding(df):
    df = df.copy()
    
    df = df.loc[(df['segCode']==1) &
                 (df['Transcript'].str.contains("^\d{0,2}:\d{2}\sme", regex=True)) &
                 (df['Question'].str.len() > 20) &
                 (df['Question'].str.contains("\?|\d{0,2}:\d{2}", regex=True)) &
                 (df['Question'].str.contains(" I |you", regex=True))
                 ,]
    
    seg = pd.Series()
    
    for i in df.index:
        seglist = []
        q = "25:25 12345678901234567890" + ' ' + df.loc[i,'Question']
        t = q + ' ' + df.loc[i,'Transcript']
        
        if re.match(pattern="^\d{1,2}:\d{2}",string=t):
            nt = re.split(pattern="\s(?=\d{1,2}:\d{2}\s\S{4,}\s)|\s(?=\S{0,3}\d{2}\sme\s)", string=t)
            for k in nt:
                seglist.append(re.split(pattern="\s",string=k,maxsplit=2))

        else:
            continue


        seg.loc[i] = seglist
            
    return(seg)

In [ ]:
questionCombinerAdding(raw)

In [ ]:
qcadd = questionCombinerAdding(raw)

raw.loc[qcadd.index,'segTrans'] = qcadd

for i in raw.index:
    if raw.loc[i,'segTrans'] == '':
        raw.loc[i,'segCode'] = 1
    else:
        raw.loc[i,'segCode'] = 0

In [ ]:
print(raw.loc[raw['segCode']==1,].shape)
print(raw.loc[raw['segCode']==0,].shape)

In [ ]:
raw['segCode'].value_counts()

In [ ]:
raw['Transcript'].str.len().describe()

In [ ]:
for i in raw.loc[(raw['segCode']==1) &
                 (raw['Transcript'].str.len() < 200)
                 ,].sample(5).index:
    print('\n')
    print(i)
    print(raw.loc[i,'Entered by'])
    print(raw.loc[i,'Who answers'])
    print(raw.loc[i,'Id'])
    print(raw.loc[i,'Question Type']),
    print("QUESTION: ",raw.loc[i,'Question'])
    print()
    print(raw.loc[i,'Transcript'])

In [ ]:
raw.loc['6619','segTrans']

In [ ]:
raw.loc['6619','segTrans'][0][1] = 'patron'

In [ ]:
raw.loc['6619','segTrans'][0][1]

In [ ]:
#109 operator IDs redacted
operators = [
    
    """ ALL OPERATOR IDS REDACTED DUE TO PRIVACY CONCERNS  """
    
]

In [ ]:
if "suepray" in operators:
    print(True)

In [ ]:
p = []
for i in raw.index:
    for k in raw.loc[i,'segTrans']:
        if re.match(pattern="\d{15,}", string=k[1]):
            k[1] = 'patron'
        if re.match(pattern="\+\d{7,}", string=k[1]):
            k[1] = 'patron'
        if re.match(pattern="\d{7,}", string=k[1]):
            k[1] = 'patron'
        if re.match(pattern="^Guest\d{3,4}", string=k[1]):
            k[1] = 'patron'
            
        if k[1] in operators:
            k[1] = 'staff'    
        p.append(k[1])

In [ ]:
pd.Series(p).value_counts()

In [ ]:
for i in raw.index:
    for k in raw.loc[i,'segTrans']:
        if (k[1] != 'staff') & (k[1] != 'patron'):
            raw.loc[i,'segCode'] = 1

In [ ]:
def newSegmenterDigits(df):
    df = df.copy()
    
    seg = pd.Series()
    
    for i in df.index:
        t = df.loc[i,'Transcript']
        
        seglist = []
            
        if re.match(pattern="^\d{1,2}:\d{2}\s\d{6,}\s|^\d{1,2}:\d{2}\sme\s",string=t):
            nt = re.split(pattern="\s(?=\d{1,2}:\d{2}\s\d{6,}\s)|\s(?=\d{1,2}:\d{2}\sme\s)", string=t)
            for k in nt:
                seglist.append(re.split(pattern="\s",string=k,maxsplit=2))
            
            
        else:
            continue
         
        seg.loc[i] = seglist
        
        
    return(seg)

In [ ]:
nsdi = newSegmenterDigits(raw[raw['segCode']>0])

raw.loc[nsdi.index,'segTrans'] = nsdi

for i in raw.index:
    if raw.loc[i,'segTrans'] == '':
        raw.loc[i,'segCode'] = 1
    else:
        raw.loc[i,'segCode'] = 0

In [ ]:
# newSegmenterDigits(raw[raw['segCode']>0])
p = []
for i in raw.index:
    for k in raw.loc[i,'segTrans']:
        if re.match(pattern="\d{15,}", string=k[1]):
            k[1] = 'patron'
        if re.match(pattern="\+\d{7,}", string=k[1]):
            k[1] = 'patron'
        if re.match(pattern="\d{7,}", string=k[1]):
            k[1] = 'patron'
        if re.match(pattern="^Guest\d{3,4}", string=k[1]):
            k[1] = 'patron'
            
        if k[1] in operators:
            k[1] = 'staff'    
        p.append(k[1])

In [ ]:
pd.Series(p).value_counts()

In [ ]:
problems = []
for i in raw.index:
    for k in raw.loc[i,'segTrans']:
        if (k[1] != 'staff') & (k[1] != 'patron'):
            problems.append(i)
            continue
problems = np.unique(problems)

In [ ]:
problems

In [ ]:
for i in problems:
    for k in raw.loc[i,'segTrans']:
        print(k)
    print('\n')

In [ ]:
for i in raw.index:
    for k in raw.loc[i,'segTrans']:
        if re.match(pattern="^me", string=k[1]):
            print(i)
            print(raw.loc[i,'segTrans'])
            print(raw.loc[i,'READ'])
            print('\n')

In [ ]:
raw['segCode'].value_counts()

In [ ]:
raw.columns

In [ ]:
raw['TransLength'].describe()

In [ ]:
raw['segCode'].value_counts()

In [ ]:
raw.loc[raw['segCode']==1,].shape

In [ ]:
raw.loc[raw['segCode']>0,['Id','Who answers','Question','Answer','Transcript','segTrans','TransLength','segCode']].to_csv("errorNotes.csv")

In [ ]:
def anotherCombinerPartial(df):
    
    manualList = [
        1730574,
        1818642,
        1956449,
        3982553,
        4855425,
        5327849,
        15968272,
        15975821,
        20067306,
        29023598,
        29574714,
        29757550,
        39263779,
    ]
    
    df = df.copy()
    
    seg = pd.Series()
    
    for i in df[df['segCode']==0].index:
        if df.loc[i,'segTrans'][0][1] == 'staff':
            q = df.loc[i,'Question']
            l = df.loc[i,'segTrans']
            if re.match(pattern="^\d{1,2}:\d{2}\s\d{6,}\s|^:\d{2}\s\d{6,}\s|^\d{2}\s\d{6,}\s", string=q):
                st = re.split(pattern="\s", string=q, maxsplit=2)
                st[1] = 'patron'
                l.insert(0,st)
                seg.loc[i] = l
            
            if re.match(pattern="^\d{1,2}:\d{2}\s\S{2,}\s", string=q):
                st = re.split(pattern="\s", string=q, maxsplit=2)
                st[1] = 'patron'
                l.insert(0,st)
                seg.loc[i] = l
                
            if re.match(pattern="^\d{6,}\s", string=q):
                st = re.split(pattern="\s", string=q, maxsplit=1)
                st[0] = 'patron'
                st.insert(0,'00:00')
                l.insert(0,st)
                seg.loc[i] = l
            
            if re.match(pattern="^could\s|^is\s|^if\s|^hey\s|^hello\S{0,1}\s|^hi\S{0,1}\s|^hello\s|^does\s|^do\s|^how\s|^i\s|^i'm\s|^who\s|^where\s|^what\s|^can\s|^why\s", string=q, flags=re.IGNORECASE):
                st = ['00:00','patron',q]
                l.insert(0,st)
                seg.loc[i] = l
            
            if df.loc[i,'Id'] in manualList:
                st = ['00:00','patron',q]
                l.insert(0,st)
                seg.loc[i] = l
            
            
            
            else:
                continue
        else:
            continue

    return(seg)

In [ ]:
anotherCombinerPartial(raw)

In [ ]:
acp = anotherCombinerPartial(raw)

raw.loc[acp.index,'segTrans'] = acp

for i in raw.index:
    if raw.loc[i,'segTrans'] == '':
        raw.loc[i,'segCode'] = 1
    else:
        raw.loc[i,'segCode'] = 0

In [ ]:
for i in raw[raw['segCode']==0].index:
    if raw.loc[i,'segTrans'][0][1] == 'staff':
#         print(raw.loc[i,'Transcript'])
#         for j in raw.loc[i,'segTrans'][:5]:
#             print(j)
        print(raw.loc[i,'Question'])
        print(raw.loc[i,'Id'])
        print('\n')

In [ ]:
counter = 0
ids = []
for i in raw.index:
    for k in raw.loc[i,'segTrans'][0:1]:
        try:
            if re.search(pattern="transfer from",flags=re.IGNORECASE,string=k[2]):
                if len(re.split(string=k[2],pattern='\s')) == 3:
                    ids.append(i)
                    counter += 1
                    continue
        except:
            continue
            
mod = []
for i in raw.loc[ids,'segTrans'].index:
    mod.append(raw.loc[i,'segTrans'][1:])
raw.loc[ids,'segTrans'] = mod    

In [ ]:
counter = 0
ids = []
for i in raw.index:
    for k in raw.loc[i,'segTrans']:
        try:
            if re.search(pattern="transfer from",flags=re.IGNORECASE,string=k[2]):
                if len(re.split(string=k[2],pattern='\s')) == 3:
#                     print(raw.loc[i,'segTrans'])
#                     print('\n')
                    k[2] = str("")
#                     print(raw.loc[i,'segTrans'])
#                     print('\n')
                    ids.append(i)
                    counter += 1
                    continue
        except:
            continue

In [ ]:
counter = 0
ids = []
for i in raw.index:
    for k in raw.loc[i,'segTrans']:
        try:
            if re.search(pattern="has left the conversation",flags=re.IGNORECASE,string=k[2]):
                if len(re.split(string=k[2],pattern='\s')) == 4:
#                     print(raw.loc[i,'segTrans'])
#                     print('\n')
                    k[2] = str("")
#                     print(raw.loc[i,'segTrans'])
#                     print('\n')
                    ids.append(i)
                    counter += 1
                    continue
        except:
            continue

In [ ]:
counter = 0
ids = []
for i in raw.index:
    for k in raw.loc[i,'segTrans']:
        try:
            if re.search(pattern="answered by",flags=re.IGNORECASE,string=k[2]):
                if len(re.split(string=k[2],pattern='\s')) == 3:
#                     print(raw.loc[i,'segTrans'])
#                     print('\n')
                    k[2] = str("")
#                     print(raw.loc[i,'segTrans'])
#                     print('\n')
                    ids.append(i)
                    counter += 1
                    continue
        except:
            continue

In [ ]:
counter = 0
ids = []
for i in raw.index:
    for k in raw.loc[i,'segTrans']:
        try:
            if re.search(pattern="transferring to \w+\.{3} please wait",flags=re.IGNORECASE,string=k[2]):
                if len(re.split(string=k[2],pattern='\s')) == 5:
#                     print(raw.loc[i,'segTrans'])
#                     print('\n')
                    k[2] = str("")
#                     print(raw.loc[i,'segTrans'])
#                     print('\n')
                    ids.append(i)
                    counter += 1
                    continue
        except:
            continue

In [ ]:
counter = 0
ids = []
for i in raw.index:
    for k in raw.loc[i,'segTrans']:
        try:
            if re.search(pattern="continued from",flags=re.IGNORECASE,string=k[2]):
                if len(re.split(string=k[2],pattern='\s')) == 3:
#                     print(raw.loc[i,'segTrans'])
#                     print('\n')
                    k[2] = str("")
#                     print(raw.loc[i,'segTrans'])
#                     print('\n')
#                     ids.append(i)
                    counter += 1
                    continue
        except:
            continue

In [ ]:
counter = 0
ids = []
for i in raw.index:
    for k in raw.loc[i,'segTrans']:
        try:
            if re.search(pattern="this is a continuing conversation",flags=re.IGNORECASE,string=k[2]):
                if len(re.split(string=k[2],pattern='\s')) == 8:
#                     print(raw.loc[i,'segTrans'])
#                     print('\n')
                    k[2] = str("")
#                     print(raw.loc[i,'segTrans'])
#                     print('\n')
                    ids.append(i)
                    counter += 1
                    continue
        except:
            continue
counter

In [ ]:
counter = 0
ids = []
for i in raw.index:
    for k in raw.loc[i,'segTrans']:
        try:
            if re.search(pattern="my name is\s\S+",flags=re.IGNORECASE,string=k[2]):
                k[2] = re.sub(pattern="my name is\s\S+",flags=re.IGNORECASE,string=k[2],repl=" my name is nameredacted ")
                ids.append(i)
                counter += 1
            continue
        except:
            continue
counter

In [ ]:
counter = 0
ids = []
for i in raw.index:
    for k in raw.loc[i,'segTrans']:
        try:
            if re.search(pattern="\Will@\S+",flags=re.IGNORECASE,string=k[2]):
                k[2] = re.sub(pattern="\Will@\S+",flags=re.IGNORECASE,string=k[2],repl=" emailill ")
                ids.append(i)
                counter += 1
            continue
        except:
            continue
counter

In [ ]:
counter = 0
ids = []
for i in raw.index:
    for k in raw.loc[i,'segTrans']:
        try:
            if re.search(pattern="\S+@\S+",flags=re.IGNORECASE,string=k[2]):
                k[2] = re.sub(pattern="\S+@\S+",flags=re.IGNORECASE,string=k[2],repl=" emailredacted ")
                ids.append(i)
                counter += 1
            continue
        except:
            continue
counter

In [ ]:
counter = 0
ids = []
for i in raw.index:
    for k in raw.loc[i,'segTrans']:
        try:
            if re.search(pattern="\W\*{3,}\W|\Wxxx{3,}\W",flags=re.IGNORECASE,string=k[2]):
                k[2] = re.sub(pattern="\W\*{3,}\W|\Wxxx{3,}\W",flags=re.IGNORECASE,string=k[2],repl=" redactedinfo ")
                ids.append(i)
                counter += 1
            continue
        except:
            continue
counter

In [ ]:
for i in raw.index:
    try:
        if raw.loc[i,'segTrans'][0][2] == raw.loc[i,'segTrans'][1][2]:
            if raw.loc[i,'segTrans'][0][1] == raw.loc[i,'segTrans'][1][1]:
                raw.loc[i,'segTrans'] = raw.loc[i,'segTrans'][1:]
    except:
        continue

In [ ]:
def getPatronLines(df):
    print('Building Patrons Lines')        
    df = df.copy()
    for i in df.index:
        patronText = str()
        for k in df.loc[i,'segTrans']:
            if len(k) == 3:
                if k[1] == 'patron':
                    patronText = patronText + k[2] + ' '
        df.loc[i,'PatronTextString'] = patronText
        
        
    return(df)

In [ ]:
raw = getPatronLines(df=raw)

In [ ]:
def getStaffLines(df):
    print('Building Operator Lines')        
    df = df.copy()
    for i in df.index:
        staffText = str()
        for k in df.loc[i,'segTrans']:
            if len(k) == 3:
                if k[1] == 'staff':
                    staffText = staffText + k[2] + ' '
        df.loc[i,'StaffTextString'] = staffText
        
        
    return(df)

In [ ]:
raw = getStaffLines(df=raw)

In [ ]:
def getAllLines(df):
    print('Building All Lines')        
    df = df.copy()
    for i in df.index:
        text = str()
        for k in df.loc[i,'segTrans']:
            if len(k) == 3:
                text = text + k[2] + ' '
        df.loc[i,'AllTextString'] = text
        
        
    return(df)

In [ ]:
raw = getAllLines(df=raw)

In [ ]:
raw.to_json('full.json')